In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
# importing required modules: Python 3 used. 
import numpy as np
import pandas as pd
import torch
print('Pytorch version ', torch.__version__)
import sys
import scipy
import networkx as nx
print('networkx version ', nx.__version__)
import copy, random
import matplotlib.pyplot as plt
from glad import glad
from glad_model import glad_model
import torch.nn as nn
from metrics import report_metrics
import warnings
warnings.filterwarnings('ignore')

/Users/djlin/opt/anaconda3
Pytorch version  2.2.2
networkx version  2.6.3


In [2]:
# initializing basic params
D = 30 # Dimension of graph
sparsity = 0.3 # erdos-renyi sparsity of true precision matrix
w_max, w_min = 0.5, 0.5 # sample true precision matrix entries ~U[w_min, w_max] 
sample_size = 100 # Number of samples from the true graph
lr_glad = 0.05 # Learning rate of GLAD
use_optimizer = 'adam' # Optimizer
INIT_DIAG = 1 # Initialize as (S + theta_init_offset * I)^-1
lambda_init = 1 # Initial lambda value
L = 15 # Number of unrolled iterations

In [3]:
# initializing precision matrix
def get_sparsity_pattern(seed=None):
    if seed != None:
        np.random.seed(seed)
    prob = sparsity
    G = nx.generators.random_graphs.gnp_random_graph(D, prob, seed=seed, directed=False)
    edge_connections = nx.adjacency_matrix(G).todense() # adjacency matrix
    return edge_connections

# Get the input data and corresponding output true precision matrix for training
def get_samples(edge_connections, seed=None, u=0.1): # mean = 0
    mean_value = 0 # zero mean of Gaussian distribution
    mean_normal = np.ones(D) * mean_value
    if seed != None:
        
        np.random.seed(seed)
        
    # uniform [w_min, w_max]
    U = np.matrix(np.random.random((D, D)) * (w_max - w_min) + w_min)
    theta = np.multiply(edge_connections, U)
    # making it symmetric
    theta = (theta + theta.T)/2 + np.eye(D)
    smallest_eigval = np.min(np.linalg.eigvals(theta))
    # Just in case : to avoid numerical error in case a epsilon complex component present
    smallest_eigval = smallest_eigval.real
    # making the min eigenvalue as u
    precision_mat = theta + np.eye(D)*(u - smallest_eigval)
    print('Smallest eigenvalue = ', np.min(np.linalg.eigvals(precision_mat)))
    
    cov = np.linalg.inv(precision_mat) # avoiding the use of pinv as data not true representative of conditional independencies.
    # get the samples 
    if seed != None:
        np.random.seed(seed)
    # Sampling data from multivariate normal distribution
    data = np.random.multivariate_normal(mean=mean_normal, cov=cov, size=sample_size)
    print('data: ', data.shape, ' theta: ', theta.shape)
    return data, precision_mat # MxD, DxD

In [4]:
# from prepare_data import get_sparsity_pattern, get_samples
# # print(dir(prepare_data))
edge_connections = get_sparsity_pattern() # the adjacency matrix
X, theta_true = get_samples(edge_connections) # input data = X, output true precision matrix = theta_true
#print('Check: ', X, theta_true)

Smallest eigenvalue =  0.0999999999999993
data:  (100, 30)  theta:  (30, 30)


In [5]:
# helper functions 
USE_CUDA = False
def convert_to_torch(data, TESTING_FLAG=False, USE_CUDA=USE_CUDA):# convert from numpy to torch variable 
    if USE_CUDA == False:
        data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor)
        if TESTING_FLAG == True:
            data.requires_grad = False
    else: # On GPU
        if TESTING_FLAG == False:
            data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor).cuda()
        else: # testing phase, no need to store the data on the GPU
            data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor).cuda()
            data.requires_grad = False
    return data

def get_optimizers(model_glad):
    lrG = lr_glad
    if use_optimizer == 'adam':
        optimizer_glad = torch.optim.Adam(model_glad.parameters(), lr=lrG, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    else:
        print('Optimizer not found!')
    return optimizer_glad


def compute_NSME(Theta_p, Theta_star):
    """
    Compute the Normalized Square Mean Error (NSME) in dB.
    
    Args:
        Theta_p (np.ndarray): Predicted matrix.
        Theta_star (np.ndarray): Ground truth matrix.
        
    Returns:
        float: NSME value in dB.
    """
    # Compute the Frobenius norm squared difference
    error_norm = np.linalg.norm(Theta_p - Theta_star, 'fro')**2
    true_norm = np.linalg.norm(Theta_star, 'fro')**2

    # Compute expected values (assuming unbiased estimates)
    E_error_norm = np.mean(error_norm)
    E_true_norm = np.mean(true_norm)

    # Compute NSME in dB
    NSME = 10 * np.log10(E_error_norm / E_true_norm)
    
    return NSME

criterion_graph = nn.MSELoss()
# Consider that all the data can be shifted to GPU
X = convert_to_torch(X)
theta_true = convert_to_torch(theta_true, TESTING_FLAG=True)

In [6]:
import time
from glad import glad
from glad_model import glad_model
# Initialize the model
model_glad = glad_model(L=L, theta_init_offset=0.1, nF=3, H=3, USE_CUDA=USE_CUDA)
optimizer_glad = get_optimizers(model_glad)
# get the sample covariance matrix for GLAD: Note, adjust the mean if needed
S = torch.matmul(X.t(), X)/X.shape[0] # DxD matrix
start_time = time.time()
for epoch in range(1000):
    optimizer_glad.zero_grad()
    theta_pred, loss = glad(S, theta_true, model_glad, [D, INIT_DIAG, lambda_init, L],criterion_graph)
#    print(theta_true.shape, theta_pred.shape)
    loss.backward() # calculate the gradients
    optimizer_glad.step() # update the weights
    compare_theta = report_metrics(np.array(theta_true), theta_pred[0].detach().numpy())
    #print('epoch: ', epoch, ' loss: ', loss.detach().numpy()[0])
    if epoch %100 == 0:
        print(': Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc ')
        print( *np.around(compare_theta, 3))
# End timing
end_time = time.time()
total_time = end_time - start_time
print(compute_NSME(theta_pred[0].detach().numpy(), theta_true.numpy()))
print(f'Total Training Time: {total_time:.2f} seconds')

: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
nan 0.0 0.0 143.0 143.0 0.0 nan 0.0 0.0 0.329 0.5
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.355 0.979 0.264 80.0 143.0 217.0 0.645 0.979 0.778 0.931 0.965
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.34 0.979 0.247 75.0 143.0 212.0 0.66 0.979 0.789 0.932 0.966
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.317 0.979 0.223 68.0 143.0 205.0 0.683 0.979 0.805 0.932 0.967
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.519 1.0 0.527 154.0 143.0 297.0 0.481 1.0 0.65 0.945 0.976
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.37 1.0 0.288 84.0 143.0 227.0 0.63 1.0 0.773 0.947 0.978
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.317 0.965 0.219 69.0 143.0 202.0 

In [37]:
import time
from detect import glad_new
from glad_model import glad_model
# Initialize the model
model_glad = glad_model(L=L, theta_init_offset=0.1, nF=3, H=3, USE_CUDA=USE_CUDA)
optimizer_glad = get_optimizers(model_glad)
# get the sample covariance matrix for GLAD: Note, adjust the mean if needed
S = torch.matmul(X.t(), X)/X.shape[0] # DxD matrix
# Start timing
start_time = time.time()
for epoch in range(1000):
    optimizer_glad.zero_grad()
    theta_pred, loss = glad_new(S, theta_true, model_glad, [D, INIT_DIAG, lambda_init, L],criterion_graph)
#    print(theta_true.shape, theta_pred.shape)
    loss.backward() # calculate the gradients
    optimizer_glad.step() # update the weights
    compare_theta = report_metrics(np.array(theta_true), theta_pred[0].detach().numpy())
    #print('epoch: ', epoch, ' loss: ', loss.detach().numpy()[0])
    if epoch %100 == 0:
        print(': Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc ')
        print( *np.around(compare_theta, 3))
# End timing
end_time = time.time()
total_time = end_time - start_time
print(compute_NSME(theta_pred[0].detach().numpy(), theta_true.numpy()))
print(f'Total Training Time: {total_time:.2f} seconds')

: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
nan 0.0 0.0 127.0 127.0 0.0 nan 0.0 0.0 0.292 0.5
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.426 0.976 0.299 95.0 127.0 216.0 0.574 0.976 0.723 0.883 0.943
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.392 0.976 0.26 83.0 127.0 204.0 0.608 0.976 0.749 0.885 0.944
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.383 0.976 0.25 80.0 127.0 201.0 0.617 0.976 0.756 0.887 0.945
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.383 0.976 0.25 80.0 127.0 201.0 0.617 0.976 0.756 0.888 0.945
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.389 0.976 0.256 82.0 127.0 203.0 0.611 0.976 0.752 0.888 0.945
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.386 0.976 0.253 81.0 127

In [39]:
import numpy as np
from sklearn.covariance import GraphicalLassoCV
# === Using GraphicalLassoCV ===
# Convert covariance estimation (Xb[0]) to fit GraphicalLassoCV
S = torch.matmul(X.t(), X)/X.shape[0] # DxD matrix

# Fit GraphicalLassoCV
glasso_model = GraphicalLassoCV(cv=10)  # 5-fold cross-validation
glasso_model.fit(X)  # Fit directly on data matrix

# Extract estimated precision matrix
precision_GlassoCV = glasso_model.precision_

# Comparing with true precision matrix
compare_theta_GLASSO = report_metrics(
        theta_true, 
        precision_GlassoCV
    )
print(': Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc ')
print(f'GLASSO: {compare_theta_GLASSO}')
print(compute_NSME(precision_GlassoCV, theta_true.numpy()))

: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
GLASSO: (0.4470588235294118, 0.3700787401574803, 0.12337662337662338, 118, 127, 85, 0.5529411764705883, 0.3700787401574803, 0.44339622641509435, 0.4708478274052716, 0.6309949892627058)
-4.5896127445014105
